In [2]:
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd
import decimal
from io import StringIO
import numpy as np


def main(event, context):
    file_name = event['name']
    if file_name.find('.zip') == -1:
        storage_client = storage.Client()
        bucket = storage_client.get_bucket('mergent_data_test')

        blob = bucket.blob(file_name)
        data_str = blob.download_as_string()
        data = str(data_str,'utf-8')
        data = StringIO(data)
        table_name = get_table_name_from_file_name(file_name)
        col_names = [schema_dict[table_name][i].name for i in range(len(schema_dict[table_name]))]
        col_names.append('Junk')
        df=pd.read_csv(data, sep='|',names=col_names,index_col=False, usecols=col_names[:-1])
        df.to_csv('temp.csv',index=False)
        for col in schema_dict[table_name]:
            if col.field_type.upper() == 'NUMERIC':
                df[col.name] = df[col.name].astype(str).map(decimal.Decimal)
            elif col.field_type.upper() == 'DATE':
                df[col.name] = pd.to_datetime(df[col.name].astype(str),format='%Y%m%d').dt.date
            if col.field_type.upper() == 'STRING':
                df[col.name] = df[col.name].astype(str)

        job_config = bigquery.LoadJobConfig(
        schema=schema_dict[table_name] ,
        write_disposition="WRITE_APPEND")

        job = bq_client.load_table_from_dataframe(df, 'eng-reactor-287421.mergent.' + table_name.upper(), job_config=job_config) 

        job.result()